### 【 데이터 전용 Dataset/DataLoader 】
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset    : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치크기만큼 데이터를 추출해 주는 역할
- 데이터셋 분리 => random_split() 함수 제공 : 타겟 클래스 고려하지 않은 랜덤한 데이터 분리

[1] 모듈 로딩 및 데이터 준비 <hr>

In [35]:
# [1-1] 모듈 로딩
import torch                                        # 텐서 및 수치과학 함수들 관련 모듈
from torch.utils.data import Dataset, DataLoader    # pytorch의 데이터 로딩
from torch.utils.data import random_split           # pytorch의 데이터셋 분리 함수
import pandas as pd

In [36]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

# [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE)
testDF  = pd.read_csv(TEST_FILE)

[2] 커스텀 데이터셋 클래스 생성 및 데이터 적용 <hr>

In [37]:
# -------------------------------------------------------------------------------------
# [2-2] 커스텀 데이터셋 클래스 정의
# -------------------------------------------------------------------------------------
# 클래스이름 : ClfDataset
# 부모클래스 : Dataset
# 오버라이딩 : _ _init_ _(self)         : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
#            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
#            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
#                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
# -------------------------------------------------------------------------------------
class ClfDataset(Dataset):

    #- 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    #- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0]
    
    #- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS
    

In [38]:
# -------------------------------------------------------------------------------------
# [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
# -------------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)   ## <= trainDS, validDS 분리
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 59999,  testDS : 9999


In [39]:
# -------------------------------------------------------------------------------------
# [2-4] 학습용/검증용/테스트용 데이터셋 분리
# -------------------------------------------------------------------------------------
# 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
# 검증용   : 제대로 데이터셋에서 규칙/패턴을 찾는지 확인 용도
#           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
# 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

# 학습용 데이터셋의 개수
print(f'allDS   :{len(allDS)}개, testDS : {len(testDS)}개')

# 학습용 데이터셋 => 학습용:검증용 = 80:20
TRAIN_SIZE = int(0.8 * len(allDS))
VALID_SIZE = len(allDS) - TRAIN_SIZE

print(f'trainDS :{TRAIN_SIZE}개')
print(f'validDS :{VALID_SIZE}개')
print(f'testDS  :{len(testDS)}개')

allDS   :59999개, testDS : 9999개
trainDS :47999개
validDS :12000개
testDS  :9999개


In [40]:
# 학습용 데이터셋 분리 => random_split()
# 단점) 분류의 경우 타겟의 비율 고려되지 않음!
genSeed = torch.Generator().manual_seed(10)
trainDS, validDS = random_split(allDS,
                                [TRAIN_SIZE, VALID_SIZE],
                                generator=genSeed)

print(f'trainDS :{type(trainDS)}, {len(trainDS)}개')
print(f'validDS :{type(validDS)}, {len(validDS)}개')
print(f'testDS  :{type(testDS)}, {len(testDS)}개')

trainDS :<class 'torch.utils.data.dataset.Subset'>, 47999개
validDS :<class 'torch.utils.data.dataset.Subset'>, 12000개
testDS  :<class '__main__.ClfDataset'>, 9999개


In [41]:
# -------------------------------------------------------------------------------------
# [2-2-3] 학습용/검증용/테스트용 데이터셋 속성
# -------------------------------------------------------------------------------------
# - datasets 타입 속성
print(f"type(testDS)---------------")
print(testDS.y.shape, len(testDS.y), sep='\n')

# - dataset.Subset타입 속성
# - dataset                 속성 : 쪼개지기 전의 원본 데이터셋 정보 확인
# - indices                 속성 : 선택된 데이터의 인덱스 정보
# - dataset.data            속성 : 실제 이미지의 로우 데이터 즉, ndarray
print(f"\n{type(trainDS)}----------------")
print(trainDS.dataset,
      trainDS.indices,
      trainDS.dataset.y.shape,
      len(trainDS.indices),
      sep='\n')

print(f"\n{type(validDS)}----------------")
print(validDS.dataset,
      validDS.indices,
      validDS.dataset.y.shape,
      len(validDS.indices),
      sep='\n')

type(testDS)---------------
(9999,)
9999

<class 'torch.utils.data.dataset.Subset'>----------------
[12151, 52178, 33389, 29194, 5246, 25000, 43442, 28563, 52530, 45434, 31300, 13408, 57873, 56115, 31360, 2088, 51356, 5377, 36809, 3401, 8625, 35074, 43510, 1675, 44753, 22226, 39400, 54179, 48200, 12413, 59392, 47680, 41365, 44631, 29867, 7472, 22148, 18143, 35843, 25132, 23740, 12444, 13659, 29321, 49873, 21455, 11710, 44093, 30083, 35398, 21760, 30112, 34823, 29740, 34101, 35178, 30955, 20558, 23589, 18641, 46957, 41603, 12466, 23146, 9718, 13177, 21393, 38708, 47277, 3167, 27032, 57373, 15520, 19529, 33728, 38476, 17031, 31604, 29215, 28443, 43939, 17339, 5380, 21859, 25445, 12565, 47979, 37507, 40272, 5425, 50441, 17957, 39202, 16495, 58866, 8647, 45721, 46078, 49247, 53266, 43030, 30841, 39575, 29932, 4434, 2011, 12850, 43563, 1610, 30847, 5672, 49789, 23028, 56212, 16709, 52870, 43087, 39581, 49259, 18132, 56312, 4276, 58344, 55469, 2158, 8097, 52502, 39112, 57414, 6647, 860, 2446

In [42]:
# -------------------------------------------------------------------------------------
# 학습용/검증용/테스트용 데이터셋에 카테고리별 데이터 분포
# - 균형 데이터셋 & 불균형 데이터셋
# -------------------------------------------------------------------------------------
from collections import Counter

# 테스트 데이터셋의 카테고리별 분포
testC = Counter(testDS.y.tolist())
print("testDict ->", {f'{k}번': int((v/len(testDS))*100) for k, v in testC.items()})

# 학습용 데이터셋의 카테고리별 분포
sel_train = [trainDS.dataset.y[idx].item() for idx in trainDS.indices]
trainC = Counter(sel_train)

trainDict = dict(sorted(trainC.items()))
print("trainDict ->", {f'{k}번': int((v/len(trainDS))*100) for k, v in trainDict.items()})

# 검증용 데이터셋의 카테고리별 분포
val_train = [validDS.dataset.y[idx].item() for idx in validDS.indices]
validC = Counter(val_train)

validDict = dict(sorted(validC.items()))
print("validDict ->", {f'{k}번': int((v/len(validDS))*100) for k, v in validDict.items()})

testDict -> {'2번': 10, '1번': 11, '0번': 9, '4번': 9, '9번': 10, '5번': 8, '6번': 9, '7번': 10, '3번': 10, '8번': 9}
trainDict -> {'0번': 9, '1번': 11, '2번': 9, '3번': 10, '4번': 9, '5번': 9, '6번': 9, '7번': 10, '8번': 9, '9번': 10}
validDict -> {'0번': 10, '1번': 10, '2번': 10, '3번': 10, '4번': 10, '5번': 8, '6번': 9, '7번': 10, '8번': 9, '9번': 9}
